In [1]:
import cv2
import numpy as np
import os

# Data Collection

In [2]:
cap=cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

face_data=[]
datapath='data/'
filename=input("enter name")
while True:
    
    ret,gray_frame=cap.read()
    #gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(gray_frame,1.3,5)
    
    if len(faces)==0:
        continue
   
    #to get largest face
    faces=sorted(faces, key=lambda f:f[2]*f[3])
    for face in faces:
        x,y,w,h=face
        #bounding box
        cv2.rectangle(gray_frame,(x,y),(x+w,y+h),(0,0,0),2)
        
        #crop out roi
        offset=10
        facesec=gray_frame[y-offset:y+h+offset,x-offset:x+w+offset]
        facesec=cv2.resize(facesec,(100,100))
        face_data.append(facesec)
        print(len(facesec))
    
    cv2.imshow("frame",gray_frame)

    key=cv2.waitKey(1) & 0xFF
    
    if key==ord('q'):
        break
#face data to numpy array
face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

#save to file system
np.save(datapath+filename+'.npy',face_data)
print("Data saved successfully")

cap.release()
cv2.destroyAllWindows()

enter nameMammam
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
(171, 30000)
Data saved successfully


# Data Preparation

In [2]:
datapath="data/"
face_data=[]
labels=[]
class_id=0
name={}

for fx in os.listdir(datapath):
    
    #get name of each file
    if fx.endswith('.npy'):
        #mapping name with id
        name[class_id]=fx[:-4]
        
    #get the contents of the file
    data_item=np.load(datapath+fx)
    print(class_id,name[class_id],data_item.shape)
    
    #contains data of all files in data
    face_data.append(data_item)
    
    #creating labels for each row in face data i.e, data item
    target=class_id*np.ones((data_item.shape[0],))
    labels.append(target)
    
    class_id=class_id+1
# form face dataset with all face_data and labels for all data
face_dataset=np.concatenate(face_data,axis=0)

#reshaping labes to convert to 2d array(with 1 column)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
print("face_dataset",face_dataset.shape)
print("face_labels",face_labels.shape)

#face data and labels together
trainset=np.concatenate((face_dataset,face_labels),axis=1)
print("trainset",trainset.shape)

0 Archita (220, 30000)
1 Mammam (171, 30000)
face_dataset (391, 30000)
face_labels (391, 1)
trainset (391, 30001)


In [3]:
#after reshpe
print(face_labels.shape)

(391, 1)


In [13]:
#before reshape
print(face_labels.shape)

(391, 1)


In [5]:
print(face_dataset)
print(face_dataset.shape)

[[ 56  84 102 ...   0   0   0]
 [ 42  70  81 ...  66  60  95]
 [ 50  79  93 ...  94  80 120]
 ...
 [ 77 100 104 ... 200 210 224]
 [ 81  99 109 ... 216 214 232]
 [ 80 101 105 ... 172 176 228]]
(391, 30000)


In [6]:
print(trainset.shape) # extra 1 column of labels

(391, 30001)


In [8]:
print(trainset[219][30000]) # Archita till 219
print(trainset[390][30000]) # Mammam from 220 to 390

0.0
1.0


# KNN Algorithm

In [9]:
def distance(x1,x2):
    return np.sqrt(((x1-x2)**2).sum())

In [10]:
def knn(train,test,k=5):
    dist=[]
    
    for i in range(0,train.shape[0]):
        
        #get the vector(face data) and label
        xi=train[i,:-1]
        yi=train[i,-1]
        
        d=distance(test,xi)
        dist.append([d,yi])
    
    #sort on the basis of 1st parameter (here d)
    dist=sorted(dist,key= lambda x:x[0])
    #first k ele
    dist=dist[:k]
    
    #retreive only labels from dist
    labels=np.array(dist)[:,-1]
    
    #get frequencies of each label
    output=np.unique(labels,return_counts=True)
    
    #find max frequency and return that label
    index=np.argmax(output[1])
    return output[0][index]

# Testing (face recognition)

In [11]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face

        #Get the face ROI
        offset = 10
        facesec = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        facesec = cv2.resize(facesec,(100,100))

        #Predicted Label (out)
        out = knn(trainset,facesec.flatten())

        #Display on the screen the name and rectangle around it
        pred_name = name[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()